# Aggregating counts and rates of hospital admissions and make a table

In [1]:
import pandas as pd
import numpy as np
import functools as ft
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

## 0- functions and read data

In [2]:
def data_process(data, infection):
    #replace SMs with 5 in data and data_hosped and NaN with 0 in data_hosped 
    data = data.replace({'SM': 5})
    # data_hosped = data_hosped.replace({'SM': 5, np.nan:0})
    #conver counts into int
    data[infection] = data[infection].astype(str).replace('\.0', '', regex=True).astype(int)
    # data_hosped[infection] = data_hosped[infection].astype(str).replace('\.0', '', regex=True).astype(int)

    #add hosp rate
    data_hosped[infection+'_rate'] = (data_hosped[infection]/data[infection])*1000
    data_hosped[infection+'_rate'] = np.round(data_hosped[infection+'_rate'], decimals=1)

    #replace rate with SN if infection count equals 5
    data_hosped.loc[data_hosped[infection] == 5, infection+'_rate'] = 'SN'

    #calculate total using sex_female and sex_male
    total = data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0])][infection] + data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0])][infection]
    total = "{:,}".format(total)
    data_hosped['variable'] = data_hosped['variable'].str.replace('antibacterial_brit', 'total')
    
    #add comma to thousands of column count
    data_hosped[infection] = data_hosped[infection].apply(lambda x : "{:,}".format(x))
    #make new column with counts and rates
    data_hosped[infection+'_rate_count'] = data_hosped[infection].astype(str) + ' (' + data_hosped[infection+'_rate'].astype(str) + ')'
    
    #replace total (after making new column of counts and rates)
    data_hosped.at[int(data_hosped[data_hosped['variable']=='total'].index[0]),infection+'_rate_count'] = total

    #drop redundant columns and rename the needed column
    data_hosped = data_hosped.drop([infection, infection+'_rate'], axis=1)
    data_hosped = data_hosped.rename(columns={infection+'_rate_count': infection})
    #replace any row with no value (=0) with a dash 
    data_hosped[infection] = data_hosped[infection].replace({'0 (0.0)': '-'})
    #replace count and rate if rate contains SN
    data_hosped.loc[data_hosped[infection].str.contains('SN', regex=False), infection] = 'SN' 

    return data_hosped[[infection, 'variable']]

In [3]:
summary_table_cases_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/prob_pred_groupby_bin_lrti_no_abs_incdt.csv')
summary_table_cases_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/prob_pred_groupby_bin_lrti_no_abs_prevl.csv')

summary_table_cases_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/prob_pred_groupby_bin_urti_combined_no_abs_incdt.csv')
summary_table_cases_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/prob_pred_groupby_bin_urti_combined_no_abs_prevl.csv')

summary_table_cases_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/prob_pred_groupby_bin_uti_no_abs_incdt.csv')
summary_table_cases_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/prob_pred_groupby_bin_uti_no_abs_prevl.csv')

summary_table_cases_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/prob_pred_groupby_bin_sinusitis_no_abs_incdt.csv')
summary_table_cases_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/prob_pred_groupby_bin_sinusitis_no_abs_prevl.csv')

summary_table_cases_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/prob_pred_groupby_bin_otmedia_no_abs_incdt.csv')
summary_table_cases_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/prob_pred_groupby_bin_otmedia_no_abs_prevl.csv')

summary_table_cases_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/prob_pred_groupby_bin_ot_externa_no_abs_incdt.csv')
summary_table_cases_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/prob_pred_groupby_bin_ot_externa_no_abs_prevl.csv')

summary_table_cases_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/prob_pred_groupby_bin_urti_no_abs_incdt.csv')
summary_table_cases_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/prob_pred_groupby_bin_urti_no_abs_prevl.csv')

summary_table_cases_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/prob_pred_groupby_bin_cough_no_abs_incdt.csv')
summary_table_cases_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/prob_pred_groupby_bin_cough_no_abs_prevl.csv')

summary_table_cases_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/prob_pred_groupby_bin_cough_cold_no_abs_incdt.csv')
summary_table_cases_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/prob_pred_groupby_bin_cough_cold_no_abs_prevl.csv')

summary_table_cases_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/prob_pred_groupby_bin_throat_no_abs_incdt.csv')
summary_table_cases_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/prob_pred_groupby_bin_throat_no_abs_prevl.csv')

In [9]:
#select relevant columns of each df
summary_table_cases_lrti_incdt = summary_table_cases_lrti_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_lrti_prevl = summary_table_cases_lrti_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_urti_combined_incdt = summary_table_cases_urti_combined_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_urti_combined_prevl = summary_table_cases_urti_combined_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_uti_incdt = summary_table_cases_uti_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_uti_prevl = summary_table_cases_uti_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_sinusitis_incdt = summary_table_cases_sinusitis_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_sinusitis_prevl = summary_table_cases_sinusitis_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_otmedia_incdt = summary_table_cases_otmedia_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_otmedia_prevl = summary_table_cases_otmedia_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_ot_externa_incdt = summary_table_cases_ot_externa_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_ot_externa_prevl = summary_table_cases_ot_externa_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_urti_incdt = summary_table_cases_urti_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_urti_prevl = summary_table_cases_urti_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_cough_incdt = summary_table_cases_cough_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_cough_prevl = summary_table_cases_cough_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_cough_cold_incdt = summary_table_cases_cough_cold_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_cough_cold_prevl = summary_table_cases_cough_cold_prevl[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_throat_incdt = summary_table_cases_throat_incdt[['bins', 'pred_count', 'ab_prob_sum']]
summary_table_cases_throat_prevl = summary_table_cases_throat_prevl[['bins', 'pred_count', 'ab_prob_sum']]

In [11]:
def data_process(data, infection):
    #replace SMs with 5 in data and data_hosped and NaN with 0 in data 
    data[['pred_count', 'ab_prob_sum']] = data[['pred_count', 'ab_prob_sum']].replace({'SM': 5})
    #conver counts into int
    data[['pred_count', 'ab_prob_sum']] = data[['pred_count', 'ab_prob_sum']].astype(str).replace('\.0', '', regex=True).astype(int)

    #add hosp rate and round to have 1 decimal
    data[infection+'_ab_perc'] = (data['ab_prob_sum']/data['pred_count'])*100
    data[infection+'_ab_perc'] = np.round(data[infection+'_ab_perc'], decimals=1)

    #replace rate with SN if infection count equals 5
    data.loc[data['ab_prob_sum'] == 5, infection+'_ab_perc'] = 'SN'

    #calculate total using sex_female and sex_male
    total = data.pred_count.sum()
    total = "{:,}".format(total)
    # data_hosped['variable'] = data_hosped['variable'].str.replace('antibacterial_brit', 'total')
    
    #add comma to thousands of column count
    data['ab_prob_sum'] = data['ab_prob_sum'].apply(lambda x : "{:,}".format(x))
    #make new column with counts and rates
    data[infection+'_count_perc'] = data['ab_prob_sum'].astype(str) + ' (' + data[infection+'_ab_perc'].astype(str) + ')'

    #add new row with total
    new_row = pd.DataFrame({'bins':'total', infection+'_count_perc':total}, index=[0])
    data_total = pd.concat([new_row,data.loc[:]]).reset_index(drop=True)

    return data_total[[infection+'_count_perc']] #'bins',

## 1- process data to make a table of counts and rates, and save it

In [13]:
data_lrti_incdt = data_process(summary_table_cases_lrti_incdt, 'lrti_incdt')
data_lrti_prevl = data_process(summary_table_cases_lrti_prevl, 'lrti_prevl')
data_urti_combined_incdt = data_process(summary_table_cases_urti_combined_incdt, 'urti_combined_incdt')
data_urti_combined_prevl = data_process(summary_table_cases_urti_combined_prevl, 'urti_combined_prevl')
data_uti_incdt = data_process(summary_table_cases_uti_incdt, 'uti_incdt')
data_uti_prevl = data_process(summary_table_cases_uti_prevl, 'uti_prevl')
data_sinusitis_incdt = data_process(summary_table_cases_sinusitis_incdt, 'sinusitis_incdt')
data_sinusitis_prevl = data_process(summary_table_cases_sinusitis_prevl, 'sinusitis_prevl')
data_otmedia_incdt = data_process(summary_table_cases_otmedia_incdt, 'otmedia_incdt')
data_otmedia_prevl = data_process(summary_table_cases_otmedia_prevl, 'otmedia_prevl')
data_ot_externa_incdt = data_process(summary_table_cases_ot_externa_incdt, 'ot_externa_incdt')
data_ot_externa_prevl = data_process(summary_table_cases_ot_externa_prevl, 'ot_externa_prevl')
data_cough_incdt = data_process(summary_table_cases_cough_incdt, 'cough_incdt')
data_cough_prevl = data_process(summary_table_cases_cough_prevl, 'cough_prevl')
data_cough_cold_incdt = data_process(summary_table_cases_cough_cold_incdt, 'cough_cold_incdt')
data_cough_cold_prevl = data_process(summary_table_cases_cough_cold_prevl, 'cough_cold_prevl')
data_urti_incdt = data_process(summary_table_cases_urti_incdt, 'urti_incdt')
data_urti_prevl = data_process(summary_table_cases_urti_prevl, 'urti_prevl')
data_throat_incdt = data_process(summary_table_cases_throat_incdt, 'throat_incdt')
data_throat_prevl = data_process(summary_table_cases_throat_prevl, 'throat_prevl')

dfs =[data_lrti_incdt, data_lrti_prevl, 
          data_urti_combined_incdt, data_urti_combined_prevl,
          data_uti_incdt, data_uti_prevl,
          data_sinusitis_incdt, data_sinusitis_prevl,
          data_otmedia_incdt, data_otmedia_prevl,
          data_ot_externa_incdt, data_ot_externa_prevl,
          data_urti_incdt, data_urti_prevl,
          data_cough_incdt, data_cough_prevl,
          data_cough_cold_incdt, data_cough_cold_prevl,
          data_throat_incdt, data_throat_prevl]

#concat dfs
data_cases = pd.concat(dfs,axis=1)

# #save hosped count and rate table
data_cases.to_csv(f'../output/aggregate_cases_table/cases_count_perc_table_all_infections.csv', index=False)
data_cases

,lrti_incdt_count_perc,lrti_prevl_count_perc,urti_combined_incdt_count_perc,urti_combined_prevl_count_perc,uti_incdt_count_perc,uti_prevl_count_perc,sinusitis_incdt_count_perc,sinusitis_prevl_count_perc,otmedia_incdt_count_perc,otmedia_prevl_count_perc,ot_externa_incdt_count_perc,ot_externa_prevl_count_perc,urti_incdt_count_perc,urti_prevl_count_perc,cough_incdt_count_perc,cough_prevl_count_perc,cough_cold_incdt_count_perc,cough_cold_prevl_count_perc,throat_incdt_count_perc,throat_prevl_count_perc
0,"90,646","1,156","354,610","4,367","89,860","1,118","89,225","1,117","91,124","1,083","89,175","1,159","88,123","1,051","88,964","1,092","89,294","1,101","88,207","1,105"
1,830 (9.2),5 (SN),"3,165 (9.0)",30 (6.9),789 (8.9),10 (7.6),835 (9.3),10 (9.8),753 (8.3),8 (8.2),791 (8.9),8 (7.3),875 (9.8),7 (6.3),821 (9.2),9 (8.1),795 (9.0),14 (11.4),823 (9.3),11 (10.2)
2,803 (8.9),14 (11.7),"3,253 (9.2)",43 (9.5),786 (8.6),6 (5.3),842 (9.4),9 (8.7),843 (9.2),11 (10.0),850 (9.5),7 (6.2),758 (8.7),9 (8.9),830 (9.4),12 (10.1),841 (9.3),9 (6.7),816 (9.2),10 (8.8)
3,840 (9.3),9 (8.0),"3,263 (9.2)",38 (9.5),817 (9.0),9 (7.7),817 (9.2),9 (8.3),824 (9.0),13 (12.6),796 (9.0),16 (13.7),838 (9.4),12 (11.2),775 (8.8),9 (8.7),773 (8.8),13 (11.9),756 (8.7),7 (7.0)
4,825 (9.0),13 (11.1),"3,298 (9.3)",36 (8.7),827 (9.2),9 (8.7),827 (9.3),10 (8.5),780 (8.7),12 (10.1),788 (8.9),12 (10.3),765 (8.7),10 (7.9),801 (9.0),12 (11.5),827 (9.2),9 (9.6),815 (9.1),10 (9.0)
5,824 (9.1),13 (10.4),"3,153 (8.9)",38 (8.8),805 (9.0),5 (SN),818 (9.0),9 (7.7),827 (9.0),9 (8.0),788 (8.8),9 (8.9),795 (8.9),8 (7.8),805 (9.1),10 (10.3),795 (8.9),12 (11.5),807 (9.1),11 (10.0)
6,820 (9.1),12 (12.2),"3,273 (9.2)",37 (8.3),825 (9.1),8 (8.5),800 (9.0),11 (9.0),858 (9.4),11 (10.6),812 (9.1),14 (10.9),752 (8.6),7 (8.0),844 (9.4),15 (12.2),811 (9.1),7 (6.5),815 (9.4),11 (8.9)
7,841 (9.2),12 (8.7),"3,175 (9.0)",38 (9.1),834 (9.4),12 (12.5),783 (8.9),15 (12.0),782 (8.7),9 (8.9),807 (9.0),12 (9.9),744 (8.5),10 (7.9),824 (9.2),9 (8.6),865 (9.6),11 (10.1),797 (9.1),7 (6.7)
8,755 (8.4),6 (5.6),"3,219 (9.0)",43 (9.2),798 (8.9),11 (8.7),793 (8.9),7 (6.8),828 (9.0),6 (5.1),797 (8.9),13 (10.6),771 (8.9),8 (8.4),806 (9.0),13 (11.3),808 (8.9),11 (9.6),820 (9.3),6 (6.2)
9,794 (8.8),9 (7.8),"3,192 (9.0)",43 (9.8),805 (8.9),9 (8.8),801 (9.0),10 (10.3),883 (9.6),16 (12.9),816 (9.2),13 (11.3),835 (9.4),6 (6.8),810 (9.0),6 (5.8),800 (9.0),9 (9.3),775 (8.7),14 (11.4)
